In [ ]:
"""
Algorithm Description:

This algorithm creates the seven filter criteria as defined in the paper: Positive EPS, Market Cap, Momentum crossover, 
which uses price and the two Moving averages. Also uses  Beta, Annualized Volatility andZ-Score of PE ratio. We use our
original long criteria as mentioned in the paper.
Runs daily.

"""

import quantopian.algorithm as algo
from quantopian.pipeline import Pipeline
from quantopian.algorithm import order_optimal_portfolio
from quantopian.algorithm import attach_pipeline, pipeline_output
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.data import morningstar
from quantopian.pipeline.classifiers.fundamentals import Sector
from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline.factors import SimpleMovingAverage, SimpleBeta, AnnualizedVolatility
from quantopian.pipeline import CustomFactor
import quantopian.optimize as opt
import pandas as pd
import numpy as np

def initialize(context):
    # Schedule our rebalance function to run at the start of
    # each week, when the market opens.
    algo.schedule_function(
        rebalance,
        date_rule=algo.date_rules.every_day(), #decision: daily
        time_rule=algo.time_rules.market_open()    
    )

    # Create our dynamic stock selector.
    algo.attach_pipeline(make_pipeline(), 'pipeline')


def make_pipeline():
    
    universe = QTradableStocksUS() #Whole american stock market

    # Filter 1: EPS has to be larger than zero
    eps = Fundamentals.basic_eps_earnings_reports.latest
    eps_filter = eps > 0 

    #Filter 2: Market cap must be larger than 20th percentile
    market_cap = Fundamentals.market_cap.latest
    market_cap_filter = market_cap.percentile_between(20,100)
    
    #Filter 3 and 4: Uses crossover of price and 20 day moving average

    ma20 = SimpleMovingAverage(inputs = [USEquityPricing.close], window_length = 20, mask= universe)
    price = USEquityPricing.close.latest

    long_ma_filter = price > ma20 
    ma_filter =  long_ma_filter 
    
    #Filter 5: 1y Beta against SPY has to be below 1 (defensive stocks only)
    #SPY = symbols('SPY') ; seems like this doesnt work as it returns SPY as a list, we therefore use sid(8554)
    beta = SimpleBeta(target=sid(8554), regression_length= 252)
    beta_filter = beta < 1
    
    #Filter 6: Yearly Volatility has to be below the 85 percentile of all assets in the universe
    vola = AnnualizedVolatility(window_length=252)
    vola_filter = vola.percentile_between(0,85)

    #Filter 7: Z-Score of PE ratio
    #Creating the mean of PE grouped by sector for zscore
    class IndustryMeanPE(CustomFactor):  
        inputs = [Fundamentals.pe_ratio, Fundamentals.morningstar_sector_code]  
        window_length = 252

        def compute(self, today, assets, out, pe, industry_codes):  
            df = pd.DataFrame(index=assets, data={"pe_ratio": pe[-1], "industry_codes": industry_codes[-1]}) 

            out[:] = df.groupby("industry_codes").transform(np.mean).values.flatten()
        
    #Creating the standard deviation of the PE grouped by sector for zscore
    class industry_std(CustomFactor):  
        inputs = [Fundamentals.pe_ratio, Fundamentals.morningstar_sector_code]  
        window_length = 252

        def compute(self, today, assets, out, pe, industry_codes):  
            df = pd.DataFrame(index=assets, data={"pe_ratio": pe[-1], "industry_codes": industry_codes[-1]}) 

            out[:] = df.groupby("industry_codes").transform(np.nanstd).values.flatten()
        
    #Getting PE ratio and calulating zscore
    pe = Fundamentals.pe_ratio.latest
    industry_pe = IndustryMeanPE()
    industry_std_of_pe = industry_std()
    zscore = (pe - industry_pe)/industry_std_of_pe
    
    #We only want below average PE ratios so zscore < 0
    zscore_filter = zscore <0 #means below average pe ratio
    
    #All combined
    positive_ma =   eps_filter & market_cap_filter & ma_filter & beta_filter & vola_filter & zscore_filter
 #     positive_ma = eps_filter & market_cap_filter & vola_filter &  beta_filter & pe_filter & zscore_filter & bollinger_filter
    return Pipeline(
        columns={
            'positive_ma' : positive_ma,        
        },
    screen = universe&pe.notnull()&industry_pe.notnull())


def compute_target_weights(context, data):
    
    #Here we want to compute our ordering weights
    
        # Initialize empty target weights dictionary.
    # This will map securities to their target weight.
    weights = {}

    # compute even target weights for each security.
    if context.positive_ma:
        long_weight = 1 / len(context.positive_ma) 

    else:
        return weights

    # Exit positions in our portfolio if they are not in our longs list.
    for security in context.portfolio.positions:
        if security not in context.positive_ma  and data.can_trade(security):
            weights[security] = 0

    for security in context.positive_ma:
        weights[security] = long_weight

    return weights


def before_trading_start(context, data):
    """
    Called every day before market open.
    """
    pipe_results = pipeline_output('pipeline')

    # These are the securities that we are interested in trading each day.
    context.positive_ma = []
    for sec in pipe_results[pipe_results['positive_ma']].index.tolist():
        if data.can_trade(sec):
            context.positive_ma.append(sec)
            

def rebalance(context, data):
    """
    Execute orders according to our schedule_function() timing.
    """
    context.max_leverage = 1.0 #no leverage
    context.max_pos_size = 0.2 #maximal position size is 20% of portfolio
    
    target_weights = compute_target_weights(context, data)
    
      # Constrain target portfolio's leverage
    max_leverage = opt.MaxGrossExposure(context.max_leverage)

      # Constrain max position Size
    constrain_pos_size = opt.PositionConcentration.with_equal_bounds(
          -context.max_pos_size,
          context.max_pos_size)
    
        
    if target_weights:
        order_optimal_portfolio(
            objective=opt.TargetWeights(target_weights),
            constraints=[constrain_pos_size, max_leverage],
        )        
        
